This notebook will encompass all calculations regarding the LV3 Recovery/eNSR Drop Test.

In [1]:
import math
import sympy
from scipy.integrate import odeint
sympy.init_printing()
%matplotlib inline

In [2]:
m= 28 # mass of payload [kg]

In [3]:
# Step a - static line extending
## Assumptions:
## static line is approximately 2m long
## plane is flying at approximately 85 mph = 38 m/s

# Variables
## P = speed of plane
## va = velocity at instant the system is pushed from the plane
## g = acceleration due to gravity
## sl = static line length
## Lva = vertical length gained from step a 
## Lha = horizaontal length gained from step a
## ta = time for step a to be complete

P = 38 # m/s
va = 0
g = 9.81 # m/s^2
sl = 2 # m

# Calculations
# vertical Distance Gained
## since the static line is 2m, assuming it falls only in the vertical direction:
Lva = 2

# horizontal Distance Gained
## speed of plane times time to drop 2m static line
# 1/2*g*ta**2 = sl
ta = math.sqrt(2*sl/g) # assuming no drag
Lha = P*ta

print('step a (from drop to static line disconnect):')
print('time to free fall fall 2 m = ', ta, ' s')
print('vertical length gained = ', Lva, ' m')
print('horizontal length gained = ', Lha, ' m')

step a (from drop to static line disconnect):
time to free fall fall 2 m =  0.638550856814101  s
vertical length gained =  2  m
horizontal length gained =  24.264932558935836  m


In [4]:
# Step b - deployment timer running
## deployment timer is a 2 sec timer

# Variables
## P = speed of plane
## vb = velocity after 2m static line (aka instant static line 'snaps')
## g = acceleration due to gravity
## Lvb = vertical length gained from step b
## Lhb = horizontal length gained from step b
## tb = time for step b to be complete

# Calculations
# vertical velocity at time of 'snap'
vb = va + (g*ta)

# since the deployment is controlled by a timer:
tb = 2

# vertical length gained
Lvb = (vb*tb) + (0.5*g*(tb**2))

# horizontal length gained
Lhb = P*tb

print('step b (from static line disconnect to timer runout):')
print('velocity at beginning of step b = ', vb, ' m/s')
print('vertical length gained = ', Lvb, ' m')
print('horizontal length gained = ', Lhb, ' m')

step b (from static line disconnect to timer runout):
velocity at beginning of step b =  6.26418390534633  m/s
vertical length gained =  32.14836781069266  m
horizontal length gained =  76  m


In [5]:
# Step c - eNSR ring separation
## Assumptions:
### the body is falling straight downward from the drogue 'chute
### drogue timer begins as ring separation occurs

# Variables
## P = speed of plane
## vc = velocity at time of ring separation
## g = acceleration due to gravity
## Lvc = vertical length gained from step c
## Lhc = horizontal length gained from step c
## tc = time for step c to be complete

# Calculations
# velocity at time of ring separation
vc = vb + g*tb

Lhc = 0
Lvc = 0

print('velocity at ring separation = ', vc, ' m/s')

velocity at ring separation =  25.884183905346333  m/s


In [6]:
# Step d - drogue line is being pulled out
## Assumptions
### no drag force considered for horizon. and vert. decent until drogue is fully unfurled
### just accounting for the 50' shock chord, therefore not including the lines coming directly from the 'chute

# Variables
## P = speed of plane
## vd = velocity after 50' shock chord is drawn out
## g = acceleration due to gravity
## Lvd = vertical distance gained from step d
## Lhd = horizontal distance gained from step d
## td = time for step d to be complete

# assuming the drogue pulls out at an angle due to a small amount of drag slowing the drogue down horizontally
## the 50' chord as the hypotenuse
# 50 = math.sqrt((x^2) + (y^2))

# vertical length gained from step d
# Lvd = vc*td + g*(td^2)

# horizontal length gained from step d
# Lhd = P*td

# calculate td by replacing x and y in the above equation
# 50**2 = (P*td)**2 + (vc*td + g*td**2)**2
Ps, vds, gs, Lvds, Lhds, tds, vcs = sympy.symbols('Ps vds gs Lvds Lhds tds vcs')
Dparms= {Ps: 50, gs: 9.81, vcs: vc}
tdEqn= (Ps*tds)**2 + (vcs*tds + gs*tds**2)**2 - 50**2
tdSolns= sympy.solve(tdEqn.subs(Dparms))
for soln in [complex(x) for x in tdSolns]:
    print('possible solution:', soln)
    if (soln.imag != 0) or (soln.real <= 0):
        print('rejected')
    else:
        print('seems fine')
        td= soln.real
#sympy.solve(tdEqn, td)

possible solution: (-0.9492191248163822+0j)
rejected
possible solution: (0.8269527246716099+0j)
seems fine
possible solution: (-2.5774176567417117-5.143086874740167j)
rejected
possible solution: (-2.5774176567417117+5.143086874740167j)
rejected


In [7]:
# now go back and calculate x and y
Lhd = P*td
Lvd = vc*td + g*(td**2)

print('time to pull out drogue:', td)
print('horizontal distance gained = ', Lhd)
print('vertical distance gained = ', Lvd)

# vertical velocity gained after the 50' drop
vd = vc + g*td

print('vertical velocity at instant line becomes taught = ', vd, 'm/s')
print('horizontal velocity: ', P)

time to pull out drogue: 0.8269527246716099
horizontal distance gained =  31.424203537521173
vertical distance gained =  28.113572841165233
vertical velocity at instant line becomes taught =  33.99659013437483 m/s
horizontal velocity:  38


In [8]:
# Step e - drogue is fully deployed
## Assumptions
### drag force in full effect
### skipping impulse and time to steady state
## Resources
### http://www.usma.edu/math/Military%20Math%20Modeling/C5.pdf
### http://www.the-rocketman.com/drogue-decent-rate.html

# cd = coeff. of drag [unitless]
# D = drag force = mass of payload*g [N]
# rho = density of air [kg/m^3]
# A = area of parachute [m^2]
# v = approx. steady state velocity of drogue [m/s]
# m = mass of payload [kg]
# w = wind speed [m/s]

D = m*g
v = 18.5 # according to rocketman
# The OpenRocket simulation says something different, but AFAIK that's using
# the drogue from LV2. -- Joe
rho = 1.2
Rd= 0.762 # drogue radius [m]
A = math.pi*(Rd**2)
cd = (2*D)/(rho*A*v**2)
m = 28
w = 12.86

# x-direction calculations
##########################

# from usma:
# mx" + cd*x' = cd*w
####### need python help here #######
# ugh, I have to go learn how to use scipy... 1 sec -- Joe
# mx" + cd*x

## homogeneous equation mx" + rho*x' = 0 
##  characteristic equation for the homogeneous differential equation is:
## mr^2 + rho*r = 0 
## where the roots are: 
## r1 = 0, r2 = -(rho/m)
## complementary solution:
## xc = C1*e^0 + C2* e^(-(rho*t/m))

## non-homogeneous equation mx" + rho*x' = rho*w
## complete solution x = C1 + C2*e^(-(rho*t/m)) + wt

## solving for C1 and C2 using results from step d as initial conditions
## except time = 0 since we are making calculations just for this step
## i.e. x(0) = x_curr_tot and vx(0) = P
## therefore C1 =  and C2 =

x_0 = Lha + Lhb + Lhc + Lhd
t = 0
vx_0 = P
C1 = Symbol('C1')
C2 = Symbol('C2')
C_1 = solve(C1 + C2*math.exp(-(rho*t/m)) + w*t - x_0, C1)
print(C_1)
C_2 = solve(C2*(-(rho/m)) + w - vx_0, C2)
print(C_2)
## NEEEED HELLLPPP should be using piecewise to solve this
## copying C_1 output from just above with the C_2 value
calc_C1 = 147.560492558936 + 586.6
print(calc_C1)

## therefore the complete solution is: 
## x = 734.1605 - 586.6*exp(-(rho*t/m)) + w*t

## if the drogue falls for 3 seconds, then
t = 3
Lhe = 734.1605 - 586.6*math.exp(-(rho*t/m)) + w*t

print('horizontal distance gained = ', Lhe, 'm')

# y-direction calculations
##########################

## from usma
## characteristic equation:
## m*r^2 + rho*r = 0
## where the roots are r = 0 and r = (-b/m)

## complementary solution:
## yc = C1 + C2*exp(-(rho*t)/m)
## solving for C1 and C2 using results from step d as initial conditions
## except time = 0 since we are making calculations just for this step

y_0 = Lva + Lvb + Lvc + Lvd
vy_0 = vd
C1 = Symbol('C1')
C2 = Symbol('C2')
## NEEEED HELLLPPP should be using piecewise to solve this
C_1 = solve(C1 + C2*math.exp(-(rho*t/m)) - y_0, C1)
print(C_1)
C_2 = solve(C2*(-(rho/m)) - vy_0, C2)
print(C_2)
## copying C_1 output from just above with the C_2 value
calc_C1 = 61.9408052275288 + (0.807117747005389*888.857809124747)
print(calc_C1)

Lve = 779.3537175364406 + (888.857809124747*math.exp(-(rho/m)*t))

print('vertical distance gained = ', Lve, 'm')

NameError: name 'Symbol' is not defined

In [168]:
# Step f - main 'chute fully deployed
## Assumptions
### drag force in full effect
### skipping impulse and time to steady state
### main 'chute is a full 18' in dia.
### after payload has gone through the drogue decent, the horizontal velocity is the same as the wind speed
## Resources
### http://www.usma.edu/math/Military%20Math%20Modeling/C5.pdf
### http://www.the-rocketman.com/decent-rate.html

# cd = coeff. of drag [unitless]
# D = drag force = weight of payload*g [N]
# rho = density of air [kg/m^3]
# A = area of parachute [m^2]
# v_main = approx. steady state velocity of main 'chute [m/s]
# m = mass of payload [kg]
# w = wind speed [m/s]

v_main = 4.83108 # according to rocketman
rho = 1.2
A = math.pi*(5.4864**2)
m = 28
D = m*g
cd = (2*D)/(rho*A*v)
w = 12.86

# x-direction calculations
##########################

# from usma:
# mx" + cd*x' = cd*w
####### need python help here #######

## homogeneous equation mx" + rho*x' = 0 
## characteristic equation for the homogeneous differential equation is:
## mr^2 + rho*r = 0 
## where the roots are: 
## r1 = 0, r2 = -(rho/m)
## complementary solution:
## xc = C1*e^0 + C2* e^(-(rho*t/m))

## non-homogeneous equation mx" + rho*x' = rho*w
## complete solution x = C1 + C2*e^(-(rho*t/m)) + wt

## solving for C1 and C2 using results from step d as initial conditions
## except time = 0 since we are making calculations just for this step
## i.e. x(0) = x_curr_tot and vx(0) = P
## therefore C1 =  and C2 =

x_0 = Lha + Lhb + Lhc + Lhd + Lhe
t = 0
vx_0 = w
C1 = Symbol('C1')
C2 = Symbol('C2')
C_1 = solve(C1 + C2*math.exp(-(rho*t/m)) + w*t - x_0, C1)
print(C_1)
C_2 = solve(C2*(-(rho/m)) + w - vx_0, C2)
print(C_2)
## NEEEED HELLLPPP should be using piecewise to solve this
## copying C_1 output from just above with the C_2 value
calc_C1 = 0 + 472.565722165574
print(calc_C1)

## therefore the complete solution is: 
## x = 472.565722165574 - 0 + w*t

## time for the main to drop approximately 1000ft = 304.8m
d = 304.8
t = 304.8/v
Lhf = 472.565722165574 + w*t

print('horizontal distance gained = ', Lhf, 'm')

# y-direction calculations
##########################

## from usma
## characteristic equation:
## m*r^2 + rho*r = 0
## where the roots are r = 0 and r = (-b/m)

## complementary solution:
## yc = C1 + C2*exp(-(rho*t)/m)
## solving for C1 and C2 using results from step d as initial conditions
## except time = 0 since we are making calculations just for this step

y_0 = Lva + Lvb + Lvc + Lvd + Lve
vy_0 = v_drogue
C1 = Symbol('C1')
C2 = Symbol('C2')
## NEEEED HELLLPPP should be using piecewise to solve this
C_1 = solve(C1 + C2*math.exp(-(rho*t/m)) - y_0, C1)
print(C_1)
C_2 = solve(C2*(-(rho/m)) - vy_0, C2)
print(C_2)
## copying C_1 output from just above with the C_2 value
calc_C1 = (0.0669425369539634*431.666666666667) + 1558.70743507288
print(calc_C1)

Lvf = 1587.6042968580075 +  (431.666666666667*math.exp(-(rho/m)*t))

print('vetical distance gained = ', Lvf, 'm')

[-C2 + 404.47384295661]
[0.0]
472.565722165574
horizontal distance gained =  1283.9221890425456 m
[-0.0669425369539634*C2 + 1622.91230333435]
[-431.666666666667]
1587.6042968580075
vetical distance gained =  1616.501158643135 m


In [169]:
## TOTALLLLSSSSS!!!!!!!!!!!!!!!!1111111111111111!!!!!!!!!!!!

# TOTAL HORIZONTAL DISTANCE TRAVELED
X_TOT = Lha + Lhb + Lhc + Lhd + Lhe + Lhf
print('TOTAL HORIZONTAL DISTANCE TRAVELED', X_TOT, 'm')

# TOTAL VERTICAL DISTANCE DESCENDED
Y_TOT = Lva + Lvb + Lvc + Lvd + Lve + Lvf
print('TOTAL VERTICAL DISTANCE DESCENDED', Y_TOT, 'm')

TOTAL HORIZONTAL DISTANCE TRAVELED 1688.3960319991556 m
TOTAL VERTICAL DISTANCE DESCENDED 3239.413461977487 m
